https://github.com/huseinzol05/NLP-Models-Tensorflow/tree/master/spelling-correction

In [4]:
# data from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/utils/helpers.py
# reuploaded to husein's S3
!wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/counts_1grams.txt

--2020-01-21 11:04:17--  https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/counts_1grams.txt
Resolving malaya-dataset.s3-ap-southeast-1.amazonaws.com (malaya-dataset.s3-ap-southeast-1.amazonaws.com)... 52.219.48.47
Connecting to malaya-dataset.s3-ap-southeast-1.amazonaws.com (malaya-dataset.s3-ap-southeast-1.amazonaws.com)|52.219.48.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9535647 (9.1M) [text/plain]
Saving to: ‘counts_1grams.txt’

counts_1grams.txt   100%[===================>]   9.09M  3.41MB/s    in 2.7s    

2020-01-21 11:04:21 (3.41 MB/s) - ‘counts_1grams.txt’ saved [9535647/9535647]



In [0]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [0]:
with open('counts_1grams.txt') as fopen:
    f = fopen.read().split('\n')[:-1]
    
words = {}
for l in f:
    w, c = l.split('\t')
    c = int(c)
    words[w] = c + words.get(w, 0)

https://github.com/cbaziotis/ekphrasis

In [0]:
!pip install ekphrasis

In [0]:
# original from https://github.com/cbaziotis/ekphrasis/blob/master/ekphrasis/classes/spellcorrect.py
# improved it

import re
from collections import Counter

class SpellCorrector:
    """
    The SpellCorrector extends the functionality of the Peter Norvig's
    spell-corrector in http://norvig.com/spell-correct.html
    """

    def __init__(self):
        """
        :param corpus: the statistics from which corpus to use for the spell correction.
        """
        super().__init__()
        self.WORDS = words
        self.N = sum(self.WORDS.values())
        
    @staticmethod
    def tokens(text):
        return REGEX_TOKEN.findall(text.lower())

    def P(self, word):
        """
        Probability of `word`.
        """
        return self.WORDS[word] / self.N

    def most_probable(self, words):
        _known = self.known(words)
        if _known:
            return max(_known, key=self.P)
        else:
            return []

    @staticmethod
    def edit_step(word):
        """
        All edits that are one edit away from `word`.
        """
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        """
        All edits that are two edits away from `word`.
        """
        return (e2 for e1 in self.edit_step(word)
                for e2 in self.edit_step(e1))

    def known(self, words):
        """
        The subset of `words` that appear in the dictionary of WORDS.
        """
        return set(w for w in words if w in self.WORDS)

    def edit_candidates(self, word, assume_wrong=False, fast=True):
        """
        Generate possible spelling corrections for word.
        """

        if fast:
            ttt = self.known(self.edit_step(word)) or {word}
        else:
            ttt = self.known(self.edit_step(word)) or self.known(self.edits2(word)) or {word}
        
        ttt = self.known([word]) | ttt
        return list(ttt)

In [0]:
corrector = SpellCorrector()

In [61]:
possible_states = corrector.edit_candidates('eting')
possible_states

['eing',
 'reting',
 'ering',
 'epting',
 'etling',
 'eling',
 'meting',
 'enting',
 'eying',
 'edting',
 'ewing',
 'elting',
 'ting',
 'etang',
 'eating',
 'kting',
 'sting',
 'eking',
 'etin',
 'beting',
 'eting',
 'geting',
 'ating',
 'ebing',
 'etting']

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [0]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [14]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 5.7MB/s 


In [15]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np

In [16]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [17]:
text = 'scientist suggests eting berger can lead to obesity'
text_mask = text.replace('eting', '**mask**')
text_mask

'scientist suggests **mask** berger can lead to obesity'

In [0]:
def tokens_to_masked_ids(tokens, mask_ind):
    masked_tokens = tokens[:]
    masked_tokens[mask_ind] = "[MASK]"
    masked_tokens = ["[CLS]"] + masked_tokens + ["[SEP]"]
    masked_ids = tokenizer.convert_tokens_to_ids(masked_tokens)
    return masked_ids

In [0]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [0]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [67]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [68]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [69]:
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, BERT_INIT_CHKPNT)

INFO:tensorflow:Restoring parameters from uncased_L-12_H-768_A-12/bert_model.ckpt


In [70]:
replaced_masks = [text_mask.replace('**mask**', state) for state in possible_states]
replaced_masks

['scientist suggests eing berger can lead to obesity',
 'scientist suggests reting berger can lead to obesity',
 'scientist suggests ering berger can lead to obesity',
 'scientist suggests epting berger can lead to obesity',
 'scientist suggests etling berger can lead to obesity',
 'scientist suggests eling berger can lead to obesity',
 'scientist suggests meting berger can lead to obesity',
 'scientist suggests enting berger can lead to obesity',
 'scientist suggests eying berger can lead to obesity',
 'scientist suggests edting berger can lead to obesity',
 'scientist suggests ewing berger can lead to obesity',
 'scientist suggests elting berger can lead to obesity',
 'scientist suggests ting berger can lead to obesity',
 'scientist suggests etang berger can lead to obesity',
 'scientist suggests eating berger can lead to obesity',
 'scientist suggests kting berger can lead to obesity',
 'scientist suggests sting berger can lead to obesity',
 'scientist suggests eking berger can lead

In [71]:
tokens = tokenizer.tokenize(replaced_masks[0])
input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
input_ids

[[101, 103, 6083, 16417, 2290, 16758, 2064, 2599, 2000, 24552, 102],
 [101, 7155, 103, 16417, 2290, 16758, 2064, 2599, 2000, 24552, 102],
 [101, 7155, 6083, 103, 2290, 16758, 2064, 2599, 2000, 24552, 102],
 [101, 7155, 6083, 16417, 103, 16758, 2064, 2599, 2000, 24552, 102],
 [101, 7155, 6083, 16417, 2290, 103, 2064, 2599, 2000, 24552, 102],
 [101, 7155, 6083, 16417, 2290, 16758, 103, 2599, 2000, 24552, 102],
 [101, 7155, 6083, 16417, 2290, 16758, 2064, 103, 2000, 24552, 102],
 [101, 7155, 6083, 16417, 2290, 16758, 2064, 2599, 103, 24552, 102],
 [101, 7155, 6083, 16417, 2290, 16758, 2064, 2599, 2000, 103, 102]]

In [72]:
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
tokens_ids

[7155, 6083, 16417, 2290, 16758, 2064, 2599, 2000, 24552]

In [0]:
def get_score(mask):
    tokens = tokenizer.tokenize(mask)
    input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
    preds = sess.run(tf.nn.softmax(model.logits), feed_dict = {model.X: input_ids})
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    return np.prod([preds[i, i + 1, x] for i, x in enumerate(tokens_ids)])

In [0]:
def generate_ids(mask):
    tokens = tokenizer.tokenize(mask)
    input_ids = [tokens_to_masked_ids(tokens, i) for i in range(len(tokens))]
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    return tokens, input_ids, tokens_ids

In [0]:
ids = [generate_ids(mask) for mask in replaced_masks]
tokens, input_ids, tokens_ids = list(zip(*ids))

In [0]:
indices, ids = [], []
for i in range(len(input_ids)):
    indices.extend([i] * len(input_ids[i]))
    ids.extend(input_ids[i])

In [77]:
ids[0]

[101, 103, 6083, 16417, 2290, 16758, 2064, 2599, 2000, 24552, 102]

In [78]:
masked_padded = tf.keras.preprocessing.sequence.pad_sequences(ids,padding='post')
masked_padded.shape

(221, 11)

In [79]:
preds = sess.run(tf.nn.log_softmax(model.logits), feed_dict = {model.X: masked_padded})
preds.shape

(221, 11, 30522)

In [80]:
indices = np.array(indices)
scores = []

for i in range(len(tokens)):
    filter_preds = preds[indices == i]
    total = np.sum([filter_preds[k, k + 1, x] for k, x in enumerate(tokens_ids[i])])
    scores.append(total)
    
scores

[-65.73369,
 -63.164936,
 -62.57653,
 -73.19431,
 -73.97393,
 -63.397663,
 -60.225082,
 -62.37426,
 -61.714867,
 -70.874214,
 -62.557587,
 -67.8916,
 -67.46682,
 -62.33689,
 -45.84125,
 -73.77844,
 -63.562042,
 -57.582092,
 -67.74833,
 -69.86494,
 -71.33391,
 -70.08538,
 -67.535416,
 -73.42106,
 -67.14624]

In [81]:
prob_scores = np.array(scores) / np.sum(scores)
prob_scores

array([0.03995042, 0.03838924, 0.03803163, 0.04448471, 0.04495853,
       0.03853068, 0.0366025 , 0.03790869, 0.03750794, 0.04307464,
       0.03802011, 0.04126192, 0.04100376, 0.03788598, 0.02786056,
       0.04483972, 0.03863058, 0.0349962 , 0.04117485, 0.04246124,
       0.04335402, 0.04259522, 0.04104545, 0.04462251, 0.04080892],
      dtype=float32)

In [82]:
probs = list(zip(possible_states, prob_scores))
probs.sort(key = lambda x: x[1])  
probs

[('eating', 0.027860561),
 ('eking', 0.034996197),
 ('meting', 0.036602505),
 ('eying', 0.03750794),
 ('etang', 0.037885983),
 ('enting', 0.037908692),
 ('ewing', 0.03802011),
 ('ering', 0.038031626),
 ('reting', 0.038389236),
 ('eling', 0.038530678),
 ('sting', 0.038630582),
 ('eing', 0.039950423),
 ('etting', 0.04080892),
 ('ting', 0.041003756),
 ('ating', 0.041045446),
 ('etin', 0.041174848),
 ('elting', 0.041261923),
 ('beting', 0.04246124),
 ('geting', 0.042595215),
 ('edting', 0.04307464),
 ('eting', 0.043354023),
 ('epting', 0.04448471),
 ('ebing', 0.044622514),
 ('kting', 0.044839717),
 ('etling', 0.044958528)]